In [61]:
spark = None
# make sure no conflicting variables 

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf # for user defined fucntions

In [62]:
spark = SparkSession.builder.appName("Stock Analysis").getOrCreate()
path_to_files= 'Stock Analysis project /StockData'

In [63]:
stocks_df = spark.read.csv(path_to_files, header=True, inferSchema=True)
# I want our headers as is from csv so we set header = True

In [64]:
stocks_df.show()

+------+----------+----------+-------+--------+--------+--------+
|Ticker|      Date|Close/Last| Volume|    Open|    High|     Low|
+------+----------+----------+-------+--------+--------+--------+
| BRK-B|05/31/2023|  $321.08 |6175417|$321.12 |$322.41 |$319.39 |
| BRK-B|05/30/2023|  $322.19 |3232461|$321.86 |$322.47 |$319.00 |
| BRK-B|05/26/2023|  $320.60 |3229873|$320.44 |$322.63 |$319.67 |
| BRK-B|05/25/2023|  $319.02 |4251935|$320.56 |$320.56 |$317.71 |
| BRK-B|05/24/2023|  $320.20 |3075393|$322.71 |$323.00 |$319.56 |
| BRK-B|05/23/2023|  $323.11 |4031342|$328.19 |$329.27 |$322.97 |
| BRK-B|05/22/2023|  $329.13 |2763422|$330.75 |$331.49 |$328.35 |
| BRK-B|05/19/2023|  $330.39 |4323538|$331.00 |$333.94 |$329.12 |
| BRK-B|05/18/2023|  $329.76 |2808329|$326.87 |$329.98 |$325.85 |
| BRK-B|05/17/2023|  $327.39 |3047626|$325.02 |$328.26 |$324.82 |
| BRK-B|05/16/2023|  $323.75 |2139996|$322.46 |$324.69 |$322.36 |
| BRK-B|05/15/2023|  $323.53 |2191609|$322.89 |$323.83 |$320.13 |
| BRK-B|05

In [65]:
stocks_df.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [66]:
stocks_df.select('Ticker', 'Date').show(5)

+------+----------+
|Ticker|      Date|
+------+----------+
| BRK-B|05/31/2023|
| BRK-B|05/30/2023|
| BRK-B|05/26/2023|
| BRK-B|05/25/2023|
| BRK-B|05/24/2023|
+------+----------+
only showing top 5 rows



Now we select microsoft 'MSFT' from Ticker and show forst 10

In [67]:
stocks_df.filter(stocks_df.Ticker == 'MSFT').show(10)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
|  MSFT|05/30/2023|  $331.21 |29503070|$335.23 |$335.74 |$330.52 |
|  MSFT|05/26/2023|  $332.89 |36630630|$324.02 |$333.40 |$323.88 |
|  MSFT|05/25/2023|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
|  MSFT|05/24/2023|  $313.85 |23384890|$314.73 |$316.50 |$312.61 |
|  MSFT|05/23/2023|  $315.26 |30797170|$320.03 |$322.72 |$315.25 |
|  MSFT|05/22/2023|  $321.18 |24115660|$318.60 |$322.59 |$318.01 |
|  MSFT|05/19/2023|  $318.34 |27546700|$316.74 |$318.75 |$316.37 |
|  MSFT|05/18/2023|  $318.52 |27275990|$314.53 |$319.04 |$313.72 |
|  MSFT|05/17/2023|  $314.00 |24315010|$312.29 |$314.43 |$310.74 |
+------+----------+----------+--------+--------+--------+--------+
only showing top 10 rows



Advanced filtering using multiple conditions

In [68]:
stocks_df.filter( (stocks_df.Ticker ==  'MSFT') & (stocks_df.Date == '05/25/2023') ).show()

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/25/2023|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
+------+----------+----------+--------+--------+--------+--------+



In [69]:
stocks_df.filter( ((stocks_df.Ticker ==  'MSFT') | (stocks_df.Ticker ==  'V' )) & (stocks_df.Date == '05/25/2023') ).show()

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/25/2023|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
|     V|05/25/2023|  $223.38 | 6827756|$222.45 |$224.69 |$220.49 |
+------+----------+----------+--------+--------+--------+--------+



In [70]:
stocks_df.filter( ( stocks_df.Ticker.isin(['MSFT', 'V', 'BRK-B','TSLA', 'SPY', 'QQQ']) ) & (stocks_df.Date == '05/25/2023') ).show()
# '.isin' provide you a list to work with 

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
| BRK-B|05/25/2023|  $319.02 | 4251935|$320.56 |$320.56 |$317.71 |
|  MSFT|05/25/2023|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
|  TSLA|05/25/2023|  $184.47 |96870720|$186.54 |$186.78 |$180.58 |
|     V|05/25/2023|  $223.38 | 6827756|$222.45 |$224.69 |$220.49 |
|   SPY|05/25/2023|    414.65|90961610|  414.74|  416.16|412.4101|
|   QQQ|05/25/2023|    339.72|66862770|  339.14|  341.01| 336.665|
+------+----------+----------+--------+--------+--------+--------+



------------------------------

We need to write a udf for converting the column Date(string) to Date(date) datatype

In [71]:
from pyspark.sql.types import DateType # import spark native datatype which is Date 
from datetime import datetime

In [72]:
date_parser = udf(lambda date: datetime.strptime(date, "%m/%d/%Y"), DateType())
# date - the argument to the lambda fucntion
# DateType(): Specifies that the output type of the UDF will be a date

# the datatime library has the fucntion strip time which is used to parse a string representing a date/ time to a "datetime" object.
# Commom formats:
# %Y: Four-digit year (e.g., 2024)
# %m: Two-digit month (01-12)
# %d: Two-digit day of the month (01-31)
# %H: Two-digit hour in 24-hour format (00-23)
# %M: Two-digit minute (00-59)
# %S: Two-digit second (00-59)
# %f: Microsecond (000000-999999)
# %p: AM or PM

In [73]:
stocks_df.withColumn('ParsedDate', date_parser(stocks_df.Date)).show(5)
# withcolumn creates a new column which is named ParsedDate and the values are coming from the udf- date_parser
# stocks_df.na.drop('column') will drop a cloumn 

+------+----------+----------+-------+--------+--------+--------+----------+
|Ticker|      Date|Close/Last| Volume|    Open|    High|     Low|ParsedDate|
+------+----------+----------+-------+--------+--------+--------+----------+
| BRK-B|05/31/2023|  $321.08 |6175417|$321.12 |$322.41 |$319.39 |2023-05-31|
| BRK-B|05/30/2023|  $322.19 |3232461|$321.86 |$322.47 |$319.00 |2023-05-30|
| BRK-B|05/26/2023|  $320.60 |3229873|$320.44 |$322.63 |$319.67 |2023-05-26|
| BRK-B|05/25/2023|  $319.02 |4251935|$320.56 |$320.56 |$317.71 |2023-05-25|
| BRK-B|05/24/2023|  $320.20 |3075393|$322.71 |$323.00 |$319.56 |2023-05-24|
+------+----------+----------+-------+--------+--------+--------+----------+
only showing top 5 rows



In [74]:
stocks_df1 = stocks_df.withColumn('ParsedDate', date_parser(stocks_df.Date))

In [75]:
stocks_df1.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- ParsedDate: date (nullable = true)



---------------------

We need to write a udf for extracing proper price in unified format

In [76]:
# fisrt, define a fucntion that will strip the string of any $ sign or improper value and leave the int/float as is

def num_parser(value):
    if isinstance(value , str):
        return float(value.strip('$')) # convert str to float and remove the '$' and return
    elif isinstance(value, int) or isinstance(value, float):
        return value
    else:
        return None

print(num_parser('$10'))
print(num_parser(10))

10.0
10


In [77]:
from pyspark.sql.types import FloatType # import spark native datatype which is `Float Type`
number_parser = udf(num_parser, FloatType())

In [78]:
stocks_df1.withColumn('Open', number_parser(stocks_df1.Open)).show()

+------+----------+----------+-------+------+--------+--------+----------+
|Ticker|      Date|Close/Last| Volume|  Open|    High|     Low|ParsedDate|
+------+----------+----------+-------+------+--------+--------+----------+
| BRK-B|05/31/2023|  $321.08 |6175417|321.12|$322.41 |$319.39 |2023-05-31|
| BRK-B|05/30/2023|  $322.19 |3232461|321.86|$322.47 |$319.00 |2023-05-30|
| BRK-B|05/26/2023|  $320.60 |3229873|320.44|$322.63 |$319.67 |2023-05-26|
| BRK-B|05/25/2023|  $319.02 |4251935|320.56|$320.56 |$317.71 |2023-05-25|
| BRK-B|05/24/2023|  $320.20 |3075393|322.71|$323.00 |$319.56 |2023-05-24|
| BRK-B|05/23/2023|  $323.11 |4031342|328.19|$329.27 |$322.97 |2023-05-23|
| BRK-B|05/22/2023|  $329.13 |2763422|330.75|$331.49 |$328.35 |2023-05-22|
| BRK-B|05/19/2023|  $330.39 |4323538| 331.0|$333.94 |$329.12 |2023-05-19|
| BRK-B|05/18/2023|  $329.76 |2808329|326.87|$329.98 |$325.85 |2023-05-18|
| BRK-B|05/17/2023|  $327.39 |3047626|325.02|$328.26 |$324.82 |2023-05-17|
| BRK-B|05/16/2023|  $323

In [79]:
stocks_df1 = (stocks_df1.withColumn("Open", number_parser(stocks_df1.Open))
                .withColumn("Close", number_parser(stocks_df1["Close/Last"]))
                .withColumn("Low", number_parser(stocks_df1.Low))
                .withColumn("High", number_parser(stocks_df1.High)))
stocks_df1.show(10)
stocks_df1.printSchema()

+------+----------+----------+-------+------+------+------+----------+------+
|Ticker|      Date|Close/Last| Volume|  Open|  High|   Low|ParsedDate| Close|
+------+----------+----------+-------+------+------+------+----------+------+
| BRK-B|05/31/2023|  $321.08 |6175417|321.12|322.41|319.39|2023-05-31|321.08|
| BRK-B|05/30/2023|  $322.19 |3232461|321.86|322.47| 319.0|2023-05-30|322.19|
| BRK-B|05/26/2023|  $320.60 |3229873|320.44|322.63|319.67|2023-05-26| 320.6|
| BRK-B|05/25/2023|  $319.02 |4251935|320.56|320.56|317.71|2023-05-25|319.02|
| BRK-B|05/24/2023|  $320.20 |3075393|322.71| 323.0|319.56|2023-05-24| 320.2|
| BRK-B|05/23/2023|  $323.11 |4031342|328.19|329.27|322.97|2023-05-23|323.11|
| BRK-B|05/22/2023|  $329.13 |2763422|330.75|331.49|328.35|2023-05-22|329.13|
| BRK-B|05/19/2023|  $330.39 |4323538| 331.0|333.94|329.12|2023-05-19|330.39|
| BRK-B|05/18/2023|  $329.76 |2808329|326.87|329.98|325.85|2023-05-18|329.76|
| BRK-B|05/17/2023|  $327.39 |3047626|325.02|328.26|324.82|2023-

In [80]:
finalStocks = stocks_df1.select(['Ticker', 'ParsedDate', 'Volume','Open', 'High','Low','Close'])
finalStocks.show()

+------+----------+-------+------+------+------+------+
|Ticker|ParsedDate| Volume|  Open|  High|   Low| Close|
+------+----------+-------+------+------+------+------+
| BRK-B|2023-05-31|6175417|321.12|322.41|319.39|321.08|
| BRK-B|2023-05-30|3232461|321.86|322.47| 319.0|322.19|
| BRK-B|2023-05-26|3229873|320.44|322.63|319.67| 320.6|
| BRK-B|2023-05-25|4251935|320.56|320.56|317.71|319.02|
| BRK-B|2023-05-24|3075393|322.71| 323.0|319.56| 320.2|
| BRK-B|2023-05-23|4031342|328.19|329.27|322.97|323.11|
| BRK-B|2023-05-22|2763422|330.75|331.49|328.35|329.13|
| BRK-B|2023-05-19|4323538| 331.0|333.94|329.12|330.39|
| BRK-B|2023-05-18|2808329|326.87|329.98|325.85|329.76|
| BRK-B|2023-05-17|3047626|325.02|328.26|324.82|327.39|
| BRK-B|2023-05-16|2139996|322.46|324.69|322.36|323.75|
| BRK-B|2023-05-15|2191609|322.89|323.83|320.13|323.53|
| BRK-B|2023-05-12|1938264|323.82|324.24|320.54|322.49|
| BRK-B|2023-05-11|2549339| 321.0|322.96|319.81|322.64|
| BRK-B|2023-05-10|2641134|326.08|326.16|320.15|

To view statistics of the cleaned data

In [81]:
finalStocks.describe(['Volume','Open', 'High','Low','Close']).show()

+-------+--------------------+------------------+------------------+------------------+------------------+
|summary|              Volume|              Open|              High|               Low|             Close|
+-------+--------------------+------------------+------------------+------------------+------------------+
|  count|               15108|             15108|             15108|             15108|             15108|
|   mean|5.1868408793685466E7|180.09656566181036| 182.1253348687101| 177.9982781513109| 180.1256089860054|
| stddev| 5.496484129953464E7|101.16125813324399|101.96625521621753|100.26590135955216|101.14891782168563|
|    min|              961133|             12.07|             12.45|              11.8|             11.93|
|    max|           914080943|            479.22|            479.98|            476.06|            477.71|
+-------+--------------------+------------------+------------------+------------------+------------------+



-----------------

Find max open price for a stock in the last 5 years

In [82]:
finalStocks.groupBy('Ticker').max('Open').withColumnRenamed("max(Open)", "Max Stock Price").show()
# .withColumnRenamed is the method of applying alias 

+------+---------------+
|Ticker|Max Stock Price|
+------+---------------+
| BRK-B|         361.39|
|  MSFT|         344.62|
|  META|         381.68|
|  TSLA|         411.47|
|  AAPL|         182.63|
|  AMZN|          187.2|
| GOOGL|         151.25|
|  NVDA|         405.95|
|     V|         250.05|
|   TSM|         141.61|
|   SPY|         479.22|
|   QQQ|         405.57|
+------+---------------+



using the inbuilt func to perform same operations

In [83]:
import pyspark.sql.functions as func
finalStocks.groupBy("Ticker").agg(func.max("Open").alias("MaxStockPrice")).show(15)

+------+-------------+
|Ticker|MaxStockPrice|
+------+-------------+
| BRK-B|       361.39|
|  MSFT|       344.62|
|  META|       381.68|
|  TSLA|       411.47|
|  AAPL|       182.63|
|  AMZN|        187.2|
| GOOGL|       151.25|
|  NVDA|       405.95|
|     V|       250.05|
|   TSM|       141.61|
|   SPY|       479.22|
|   QQQ|       405.57|
+------+-------------+



In [84]:
finalStocks.groupBy("Ticker").agg(
    func.max("Open").alias("MaxStockPrice"),
    func.sum("Volume").alias("Total Volume")
    ).show(15)

+------+-------------+------------+
|Ticker|MaxStockPrice|Total Volume|
+------+-------------+------------+
| BRK-B|       361.39|  5862401321|
|  MSFT|       344.62| 37976660472|
|  META|       381.68| 30148848043|
|  TSLA|       411.47|171802975076|
|  AAPL|       182.63|139310061360|
|  AMZN|        187.2|104503287430|
| GOOGL|       151.25| 43956560981|
|  NVDA|       405.95| 58787218324|
|     V|       250.05| 10410997871|
|   TSM|       141.61| 12506470104|
|   SPY|       479.22|107925285300|
|   QQQ|       405.57| 60437153773|
+------+-------------+------------+



Extracting Month and Year from data for further analysis

In [85]:
finalStocks = (finalStocks.withColumn("Year", func.year(finalStocks.ParsedDate))
                                .withColumn("Month", func.month(finalStocks.ParsedDate))
                                .withColumn("Day", func.dayofmonth(finalStocks.ParsedDate))
                                .withColumn("Week", func.weekofyear(finalStocks.ParsedDate))
                 )

In [86]:
finalStocks.show(10)

+------+----------+-------+------+------+------+------+----+-----+---+----+
|Ticker|ParsedDate| Volume|  Open|  High|   Low| Close|Year|Month|Day|Week|
+------+----------+-------+------+------+------+------+----+-----+---+----+
| BRK-B|2023-05-31|6175417|321.12|322.41|319.39|321.08|2023|    5| 31|  22|
| BRK-B|2023-05-30|3232461|321.86|322.47| 319.0|322.19|2023|    5| 30|  22|
| BRK-B|2023-05-26|3229873|320.44|322.63|319.67| 320.6|2023|    5| 26|  21|
| BRK-B|2023-05-25|4251935|320.56|320.56|317.71|319.02|2023|    5| 25|  21|
| BRK-B|2023-05-24|3075393|322.71| 323.0|319.56| 320.2|2023|    5| 24|  21|
| BRK-B|2023-05-23|4031342|328.19|329.27|322.97|323.11|2023|    5| 23|  21|
| BRK-B|2023-05-22|2763422|330.75|331.49|328.35|329.13|2023|    5| 22|  21|
| BRK-B|2023-05-19|4323538| 331.0|333.94|329.12|330.39|2023|    5| 19|  20|
| BRK-B|2023-05-18|2808329|326.87|329.98|325.85|329.76|2023|    5| 18|  20|
| BRK-B|2023-05-17|3047626|325.02|328.26|324.82|327.39|2023|    5| 17|  20|
+------+----

stock price for a given year

In [87]:
yearly = finalStocks.groupBy(['Ticker', 'Year']).agg(func.max("Open").alias("YearlHigh"), 
                                                        func.min("Open").alias("YearlyLow"))
yearly.show(10)

+------+----+---------+---------+
|Ticker|Year|YearlHigh|YearlyLow|
+------+----+---------+---------+
| BRK-B|2023|    331.0|   294.68|
| BRK-B|2018|    224.0|   185.43|
| BRK-B|2021|   300.88|   228.21|
| BRK-B|2020|   233.92|    165.3|
| BRK-B|2019|   227.27|   194.78|
| BRK-B|2022|   361.39|   260.58|
|  MSFT|2019|   159.45|    99.55|
|  MSFT|2021|   344.62|   212.17|
|  MSFT|2020|   229.27|   137.01|
|  MSFT|2018|   115.42|    95.14|
+------+----+---------+---------+
only showing top 10 rows



In [88]:
monthly = finalStocks.groupBy(['Ticker', 'Year', 'Month']).agg(func.max('Open').alias('Monthly High'),
                                                               func.min('Open').alias('Monthly Low'))
monthly.show()
# while performing analysis, we need to know monthly and yearly trends

+------+----+-----+------------+-----------+
|Ticker|Year|Month|Monthly High|Monthly Low|
+------+----+-----+------------+-----------+
| BRK-B|2022|   10|      297.98|     260.58|
| BRK-B|2018|    9|      222.13|     209.21|
| BRK-B|2021|   10|      290.85|     273.02|
| BRK-B|2020|   10|      216.74|     200.03|
| BRK-B|2019|    9|      212.24|     201.19|
| BRK-B|2021|    6|      292.91|      275.0|
| BRK-B|2022|   11|      317.52|     286.02|
| BRK-B|2022|    7|      297.42|      272.5|
| BRK-B|2022|    6|       316.0|     267.38|
| BRK-B|2020|    8|      218.62|     197.28|
| BRK-B|2021|   12|      300.88|     276.68|
| BRK-B|2019|    1|      204.68|     194.78|
| BRK-B|2019|    6|       213.0|     197.62|
| BRK-B|2020|    7|      195.06|     178.26|
| BRK-B|2021|    8|      291.81|     279.05|
| BRK-B|2018|    6|       196.3|     185.43|
| BRK-B|2022|    5|      325.85|     303.93|
| BRK-B|2020|   12|      231.26|     222.02|
| BRK-B|2020|    4|       194.0|      175.0|
| BRK-B|20

In [89]:
monthly = monthly.withColumn("Spread", monthly['Monthly High'] - monthly['Monthly Low'])
monthly.show()

+------+----+-----+------------+-----------+---------+
|Ticker|Year|Month|Monthly High|Monthly Low|   Spread|
+------+----+-----+------------+-----------+---------+
| BRK-B|2022|   10|      297.98|     260.58|37.400024|
| BRK-B|2018|    9|      222.13|     209.21|12.919998|
| BRK-B|2021|   10|      290.85|     273.02|17.830017|
| BRK-B|2020|   10|      216.74|     200.03|16.710007|
| BRK-B|2019|    9|      212.24|     201.19|11.050003|
| BRK-B|2021|    6|      292.91|      275.0|17.910004|
| BRK-B|2022|   11|      317.52|     286.02|     31.5|
| BRK-B|2022|    7|      297.42|      272.5|24.920013|
| BRK-B|2022|    6|       316.0|     267.38|48.619995|
| BRK-B|2020|    8|      218.62|     197.28|21.339996|
| BRK-B|2021|   12|      300.88|     276.68|24.200012|
| BRK-B|2019|    1|      204.68|     194.78| 9.899994|
| BRK-B|2019|    6|       213.0|     197.62|15.380005|
| BRK-B|2020|    7|      195.06|     178.26|16.800003|
| BRK-B|2021|    8|      291.81|     279.05| 12.76001|
| BRK-B|20

In [90]:
from pyspark.sql.functions import round

monthly.withColumn("Spread", round(monthly.Spread, 2)).show()

+------+----+-----+------------+-----------+------+
|Ticker|Year|Month|Monthly High|Monthly Low|Spread|
+------+----+-----+------------+-----------+------+
| BRK-B|2022|   10|      297.98|     260.58|  37.4|
| BRK-B|2018|    9|      222.13|     209.21| 12.92|
| BRK-B|2021|   10|      290.85|     273.02| 17.83|
| BRK-B|2020|   10|      216.74|     200.03| 16.71|
| BRK-B|2019|    9|      212.24|     201.19| 11.05|
| BRK-B|2021|    6|      292.91|      275.0| 17.91|
| BRK-B|2022|   11|      317.52|     286.02|  31.5|
| BRK-B|2022|    7|      297.42|      272.5| 24.92|
| BRK-B|2022|    6|       316.0|     267.38| 48.62|
| BRK-B|2020|    8|      218.62|     197.28| 21.34|
| BRK-B|2021|   12|      300.88|     276.68|  24.2|
| BRK-B|2019|    1|      204.68|     194.78|   9.9|
| BRK-B|2019|    6|       213.0|     197.62| 15.38|
| BRK-B|2020|    7|      195.06|     178.26|  16.8|
| BRK-B|2021|    8|      291.81|     279.05| 12.76|
| BRK-B|2018|    6|       196.3|     185.43| 10.87|
| BRK-B|2022

-----------------------

Joins

In [91]:
finalStocks.join(yearly, 
                (finalStocks.Ticker == yearly.Ticker)&(finalStocks.Year == yearly.Year),
                'inner'
                ).show()

+------+----------+-------+------+------+------+------+----+-----+---+----+------+----+---------+---------+
|Ticker|ParsedDate| Volume|  Open|  High|   Low| Close|Year|Month|Day|Week|Ticker|Year|YearlHigh|YearlyLow|
+------+----------+-------+------+------+------+------+----+-----+---+----+------+----+---------+---------+
| BRK-B|2023-05-31|6175417|321.12|322.41|319.39|321.08|2023|    5| 31|  22| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-30|3232461|321.86|322.47| 319.0|322.19|2023|    5| 30|  22| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-26|3229873|320.44|322.63|319.67| 320.6|2023|    5| 26|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-25|4251935|320.56|320.56|317.71|319.02|2023|    5| 25|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-24|3075393|322.71| 323.0|319.56| 320.2|2023|    5| 24|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-23|4031342|328.19|329.27|322.97|323.11|2023|    5| 23|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-22|2763422|3

In [92]:
historic_stocks = finalStocks.join(yearly, 
                                   (finalStocks.Ticker == yearly.Ticker)&(finalStocks.Year == yearly.Year),
                                   'inner'
                                   ).drop(yearly.Ticker, yearly.Year)

historic_stocks.show()

+----------+-------+------+------+------+------+-----+---+----+------+----+---------+---------+
|ParsedDate| Volume|  Open|  High|   Low| Close|Month|Day|Week|Ticker|Year|YearlHigh|YearlyLow|
+----------+-------+------+------+------+------+-----+---+----+------+----+---------+---------+
|2023-05-31|6175417|321.12|322.41|319.39|321.08|    5| 31|  22| BRK-B|2023|    331.0|   294.68|
|2023-05-30|3232461|321.86|322.47| 319.0|322.19|    5| 30|  22| BRK-B|2023|    331.0|   294.68|
|2023-05-26|3229873|320.44|322.63|319.67| 320.6|    5| 26|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-25|4251935|320.56|320.56|317.71|319.02|    5| 25|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-24|3075393|322.71| 323.0|319.56| 320.2|    5| 24|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-23|4031342|328.19|329.27|322.97|323.11|    5| 23|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-22|2763422|330.75|331.49|328.35|329.13|    5| 22|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-19|4323538| 331.0|333.94|329.12

In [93]:
# cond = [(historic_stocks.Ticker==monthly.Ticker) & (historic_stocks.Year == monthly.Year) & (historic_stocks.Month == monthly.Month)]
# historic_stocks = historic_stocks.join(monthly, cond, 'inner').drop(monthly.Year, historic_stocks.Ticker, monthly.Month)

In [94]:
historic_stocks = historic_stocks.join(monthly ,['Ticker', 'Year', 'Month']).show()
# using this method prevents repitition

+------+----+-----+----------+-------+------+------+------+------+---+----+---------+---------+------------+-----------+---------+
|Ticker|Year|Month|ParsedDate| Volume|  Open|  High|   Low| Close|Day|Week|YearlHigh|YearlyLow|Monthly High|Monthly Low|   Spread|
+------+----+-----+----------+-------+------+------+------+------+---+----+---------+---------+------------+-----------+---------+
| BRK-B|2023|    5|2023-05-31|6175417|321.12|322.41|319.39|321.08| 31|  22|    331.0|   294.68|       331.0|     320.44|10.559998|
| BRK-B|2023|    5|2023-05-30|3232461|321.86|322.47| 319.0|322.19| 30|  22|    331.0|   294.68|       331.0|     320.44|10.559998|
| BRK-B|2023|    5|2023-05-26|3229873|320.44|322.63|319.67| 320.6| 26|  21|    331.0|   294.68|       331.0|     320.44|10.559998|
| BRK-B|2023|    5|2023-05-25|4251935|320.56|320.56|317.71|319.02| 25|  21|    331.0|   294.68|       331.0|     320.44|10.559998|
| BRK-B|2023|    5|2023-05-24|3075393|322.71| 323.0|319.56| 320.2| 24|  21|    331.

----------------------------------

Advanced Queries

In [95]:
snapshot = finalStocks.select(['Ticker', 'ParsedDate', 'Open'])
snapshot.show()

+------+----------+------+
|Ticker|ParsedDate|  Open|
+------+----------+------+
| BRK-B|2023-05-31|321.12|
| BRK-B|2023-05-30|321.86|
| BRK-B|2023-05-26|320.44|
| BRK-B|2023-05-25|320.56|
| BRK-B|2023-05-24|322.71|
| BRK-B|2023-05-23|328.19|
| BRK-B|2023-05-22|330.75|
| BRK-B|2023-05-19| 331.0|
| BRK-B|2023-05-18|326.87|
| BRK-B|2023-05-17|325.02|
| BRK-B|2023-05-16|322.46|
| BRK-B|2023-05-15|322.89|
| BRK-B|2023-05-12|323.82|
| BRK-B|2023-05-11| 321.0|
| BRK-B|2023-05-10|326.08|
| BRK-B|2023-05-09|324.87|
| BRK-B|2023-05-08|328.26|
| BRK-B|2023-05-05|323.36|
| BRK-B|2023-05-04|323.44|
| BRK-B|2023-05-03|327.13|
+------+----------+------+
only showing top 20 rows



In [96]:
from pyspark.sql.window import Window

lag2day = Window.partitionBy("Ticker").orderBy("ParsedDate")

In [99]:
snapshot = snapshot.withColumn("2PreviosOpen", func.lag("Open", 2).over(lag2day))

In [100]:
from pyspark.sql.functions import col

snapshot.withColumn("PercentDiff", ((col("Open") - col("2PreviosOpen")) / col("2PreviosOpen")) * 100).show()


+------+----------+-----+------------+--------------------+
|Ticker|ParsedDate| Open|2PreviosOpen|         PercentDiff|
+------+----------+-----+------------+--------------------+
|  AAPL|2018-05-31|46.81|        NULL|                NULL|
|  AAPL|2018-06-01| 47.0|        NULL|                NULL|
|  AAPL|2018-06-04|47.91|       46.81|   2.349921900982327|
|  AAPL|2018-06-05|48.27|        47.0|  2.7021286335397274|
|  AAPL|2018-06-06|48.41|       47.91|  1.0436234639792172|
|  AAPL|2018-06-07|48.54|       48.27|  0.5593545788339545|
|  AAPL|2018-06-08|47.79|       48.41| -1.2807249201383943|
|  AAPL|2018-06-11|47.84|       48.54| -1.4421111449042672|
|  AAPL|2018-06-12|47.85|       47.79| 0.12554416707336016|
|  AAPL|2018-06-13|48.11|       47.84|  0.5643822259667495|
|  AAPL|2018-06-14|47.89|       47.85| 0.08359648234676038|
|  AAPL|2018-06-15|47.51|       48.11|  -1.247146707974184|
|  AAPL|2018-06-18|46.97|       47.89| -1.9210653177501873|
|  AAPL|2018-06-19|46.29|       47.51| -

In [106]:
maximumStock = Window.partitionBy("Ticker").orderBy(snapshot.Open.desc())

In [107]:
snapshot.withColumn("MaxOpen", func.row_number().over(maximumStock)).show()

+------+----------+------+------------+-------+
|Ticker|ParsedDate|  Open|2PreviosOpen|MaxOpen|
+------+----------+------+------------+-------+
|  AAPL|2022-01-04|182.63|      178.09|      1|
|  AAPL|2021-12-13|181.12|      174.91|      2|
|  AAPL|2021-12-28|180.16|      175.85|      3|
|  AAPL|2022-01-05|179.61|      177.83|      4|
|  AAPL|2021-12-30|179.47|      180.16|      5|
|  AAPL|2021-12-29|179.33|      177.09|      6|
|  AAPL|2021-12-16|179.28|      175.25|      7|
|  AAPL|2022-03-30|178.55|      172.17|      8|
|  AAPL|2021-12-31|178.09|      179.33|      9|
|  AAPL|2022-03-31|177.84|      176.69|     10|
|  AAPL|2022-01-03|177.83|      179.47|     11|
|  AAPL|2022-04-05| 177.5|      174.03|     12|
|  AAPL|2023-05-31|177.33|      173.32|     13|
|  AAPL|2021-12-27|177.09|      173.04|     14|
|  AAPL|2023-05-30|176.96|      172.41|     15|
|  AAPL|2022-03-29|176.69|      173.88|     16|
|  AAPL|2023-05-19|176.39|      171.71|     17|
|  AAPL|2022-01-12|176.12|      169.08| 